<a href="https://colab.research.google.com/github/abarb2022/Facial-Expression-Recognition-Challenge/blob/main/simple_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://youtu.be/yEXkEUqK52Q

**Downloading Kaggle data sets directly into Colab**

Install the kaggle python library

In [ ]:
! pip install kaggle

Mount the Google drive so you can store your kaggle API credentials for future use

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Make a directory for kaggle at the temporary instance location on Colab drive.

Download your kaggle API key (.json file). You can do this by going to your kaggle account page and clicking 'Create new API token' under the API section.

In [ ]:
! mkdir ~/.kaggle

If you want to copy the kaggle API credentials to the temporary location... (I recommend placing it on your Google Drive)

In [ ]:
#! cp kaggle.json ~/.kaggle/

Upload the json file to Google Drive and then copy to the temporary location.

In [ ]:
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

Change the file permissions to read/write to the owner only

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

**Competitions and Datasets are the two types of Kaggle data**

**1. Download competition data**

If you get 403 Forbidden error, you need to click 'Late Submission' on the Kaggle page for that competition.

In [ ]:
! kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

 89% 254M/285M [00:01<00:00, 126MB/s] 
100% 285M/285M [00:01<00:00, 154MB/s]


Unzip, in case the downloaded file is zipped. Refresh the files on the left hand side to update the view.

In [ ]:
! unzip challenges-in-representation-learning-facial-expression-recognition-challenge

Archive:  challenges-in-representation-learning-facial-expression-recognition-challenge.zip
  inflating: example_submission.csv  
  inflating: fer2013.tar.gz          
  inflating: icml_face_data.csv      
  inflating: test.csv                
  inflating: train.csv               


In [ ]:

import torch # Main PyTorch Library
from torch import nn # Used for creating the layers and loss function
from torch.optim import Adam # Adam Optimizer
import torchvision.transforms as transforms # Transform function used to modify and preprocess all the images
from torch.utils.data import Dataset, DataLoader # Dataset class and DataLoader for creating the objects
from sklearn.preprocessing import LabelEncoder # Label Encoder to encode the classes from strings to numbers
import matplotlib.pyplot as plt # Used for visualizing the images and plotting the training progress
from PIL import Image # Used to read the images from the directory
import pandas as pd # Used to read/create dataframes (csv) and process tabular data
import numpy as np # preprocessing and numerical/mathematical operations
import os # Used to read the images path from the directory
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from sklearn.metrics import accuracy_score, classification_report


device = "cuda" if torch.cuda.is_available() else "cpu" # detect the GPU if any, if not use CPU, change cuda to mps if you have a mac
print("Device available: ", device)


Device available:  cuda


In [ ]:
data_df = pd.read_csv('icml_face_data.csv')

# # Split data according to original splits
# train_df = data_df[data_df[' Usage'] == 'Training']
# val_df = data_df[data_df[' Usage'] == 'PublicTest']
# test_df = data_df[data_df[' Usage'] == 'PrivateTest']

 if ' Usage' not in data_df.columns:
        print("Warning: 'Usage' column not found in dataset")
        return data_df

    # Split data and remove 'Usage' column
    train_df = data_df[data_df[' Usage'] == 'Training'].drop(columns=[' Usage'])
    val_df = data_df[data_df[' Usage'] == 'PublicTest'].drop(columns=[' Usage'])
    test_df = data_df[data_df[' Usage'] == 'PrivateTest'].drop(columns=[' Usage'])

In [ ]:
class SimpleFERDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pixels = self.data.iloc[idx][' pixels']
        pixels = np.array([int(pixel) for pixel in pixels.split()], dtype=np.float32)
        image = pixels.reshape(1, 48, 48)  # 1x48x48 (channel first)

        # Simple normalization [0, 255] -> [0, 1]
        image = image / 255.0

        label = self.data.iloc[idx]['emotion']
        return torch.FloatTensor(image), torch.LongTensor([label])

In [ ]:
# Very simple CNN model
class SimpleEmotionCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(SimpleEmotionCNN, self).__init__()

        self.conv_layers = nn.Sequential(
            # 1x48x48 -> 32x24x24
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 32x24x24 -> 64x12x12
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 64x12x12 -> 128x6x6
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 6 * 6, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layers(x)
        return x

In [ ]:
train_dataset = SimpleFERDataset(train_df)
val_dataset = SimpleFERDataset(val_df)
test_dataset = SimpleFERDataset(test_df)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# # Initialize model, loss, and optimizer
# model = SimpleEmotionCNN(num_classes=7).to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
from collections import defaultdict
def train_with_history(model, train_loader, val_loader, num_epochs=15, lr=0.001):
    history = defaultdict(list)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    best_val_acc = 0.0
    best_model_weights = None

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device).squeeze()

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Store batch metrics every 50 batches
            if batch_idx % 50 == 0:
                history['batch'].append(epoch * len(train_loader) + batch_idx)
                history['train_batch_loss'].append(loss.item())
                history['train_batch_acc'].append((predicted == labels).sum().item() / labels.size(0))

        # Store epoch metrics
        avg_train_loss = train_loss / len(train_loader)
        train_acc = correct / total
        history['epoch'].append(epoch)
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_acc)

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device).squeeze()
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_acc = correct / total
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_acc)

        # Track best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_weights = model.state_dict().copy()

        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f}')

    # Store confusion matrix data
    model.eval()
    true_labels = []
    pred_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device).squeeze()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            true_labels.extend(labels.cpu().numpy())
            pred_labels.extend(predicted.cpu().numpy())

    history['confusion_matrix'] = {
        'true': true_labels,
        'pred': pred_labels,
        'class_names': ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
    }

    # Restore best model weights
    if best_model_weights:
        model.load_state_dict(best_model_weights)

    return model, history

In [ ]:
config = {
    'learning_rate': 0.001,
    'batch_size': batch_size,
    'epochs': 15,
    'architecture': 'SimpleCNN',
    'optimizer': 'Adam'
}

# Initialize and train model
model = SimpleEmotionCNN(num_classes=7).to(device)
trained_model, training_history = train_with_history(
    model,
    train_loader,
    val_loader,
    num_epochs=config['epochs'],
    lr=config['learning_rate']
)

# Test evaluation
def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device).squeeze()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

test_acc = evaluate(trained_model, test_loader)
print(f'Test Accuracy: {test_acc:.4f}')
training_history['test_acc'] = test_acc

Epoch 1/15:
Train Loss: 1.7107 | Train Acc: 0.3087
Val Loss: 1.5310 | Val Acc: 0.4048
Epoch 2/15:
Train Loss: 1.4951 | Train Acc: 0.4212
Val Loss: 1.4055 | Val Acc: 0.4581
Epoch 3/15:
Train Loss: 1.3763 | Train Acc: 0.4716
Val Loss: 1.3255 | Val Acc: 0.4896
Epoch 4/15:
Train Loss: 1.2906 | Train Acc: 0.5086
Val Loss: 1.2865 | Val Acc: 0.5018
Epoch 5/15:
Train Loss: 1.2162 | Train Acc: 0.5382
Val Loss: 1.2277 | Val Acc: 0.5286
Epoch 6/15:
Train Loss: 1.1523 | Train Acc: 0.5636
Val Loss: 1.2091 | Val Acc: 0.5378
Epoch 7/15:
Train Loss: 1.0970 | Train Acc: 0.5853
Val Loss: 1.1896 | Val Acc: 0.5478
Epoch 8/15:
Train Loss: 1.0336 | Train Acc: 0.6120
Val Loss: 1.1929 | Val Acc: 0.5481
Epoch 9/15:
Train Loss: 0.9743 | Train Acc: 0.6318
Val Loss: 1.1955 | Val Acc: 0.5528
Epoch 10/15:
Train Loss: 0.9183 | Train Acc: 0.6569
Val Loss: 1.2093 | Val Acc: 0.5598
Epoch 11/15:
Train Loss: 0.8529 | Train Acc: 0.6830
Val Loss: 1.2456 | Val Acc: 0.5539
Epoch 12/15:
Train Loss: 0.8013 | Train Acc: 0.7003


In [ ]:
!pip install wandb -q
import wandb
wandb.login()  # Follow the link to authenticate

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abarb2022 (abarb2022-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:

wandb.init(project="facial-expression-recognition", config=config)

# Log hyperparameters
wandb.config.update(config)

for epoch in range(len(training_history['epoch'])):
    wandb.log({
        'epoch': training_history['epoch'][epoch],
        'train_loss': training_history['train_loss'][epoch],
        'train_acc': training_history['train_acc'][epoch],
        'val_loss': training_history['val_loss'][epoch],
        'val_acc': training_history['val_acc'][epoch]
    })

# Log batch metrics (sampled to avoid too many points)
if 'batch' in training_history:
    batch_indices = np.linspace(0, len(training_history['batch'])-1, 1000, dtype=int)
    for idx in batch_indices:
        wandb.log({
            'batch': training_history['batch'][idx],
            'train_batch_loss': training_history['train_batch_loss'][idx],
            'train_batch_acc': training_history['train_batch_acc'][idx]
        }, commit=False)

# Log confusion matrix
if 'confusion_matrix' in training_history:
    cm_data = training_history['confusion_matrix']
    wandb.log({
        "confusion_matrix": wandb.plot.confusion_matrix(
            y_true=cm_data['true'],
            preds=cm_data['pred'],
            class_names=cm_data['class_names']
        )
    })

# Log model architecture
wandb.watch(model, log='all', log_freq=100, log_graph=True)

# Log final metrics
wandb.summary['best_val_acc'] = max(training_history['val_acc'])
wandb.summary['final_train_acc'] = training_history['train_acc'][-1]

# Save model
torch.save(trained_model.state_dict(), 'model_weights.pth')
wandb.save('model_weights.pth')

wandb.finish()

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


batch,▁
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▃▄▄▅▅▅▆▆▆▇▇▇██
train_batch_acc,▁
train_batch_loss,▁
train_loss,█▇▆▅▅▄▄▄▃▃▂▂▂▁▁
val_acc,▁▃▅▅▆▇▇▇███████
val_loss,█▅▄▃▂▁▁▁▁▁▂▂▃▄▅
batch,6686
best_val_acc,0.56255
epoch,14
